<a href="https://colab.research.google.com/github/ish-war/AI-Job-Recommendation/blob/main/AI_Job_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies

In [1]:
!pip install groq langchain langchain-community langchain-groq sentence-transformers python-dotenv faiss-cpu requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

## Install required Packages

In [2]:
# ---- Install required packages ----

import os
import json
import hashlib
import requests
from typing import List, Dict
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq

# Load environment variables (Groq key must be set)
load_dotenv()

False

## Call API keys

In [14]:
import os
from google.colab import userdata

# Load keys from Colab secrets into environment variables
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

# Check they are available
print("✅ GROQ_API_KEY is set:", bool(os.environ["GROQ_API_KEY"]))
print("✅ TAVILY_API_KEY is set:", bool(os.environ["TAVILY_API_KEY"]))


✅ GROQ_API_KEY is set: True
✅ TAVILY_API_KEY is set: True


In [16]:
# ==== Load API keys ====
load_dotenv()  # Make sure GROQ_API_KEY and TAVILY_API_KEY are in your .env file
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [17]:
import os

# Check for Groq API key
groq_key = os.getenv("GROQ_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")

if groq_key:
    print("✅ GROQ_API_KEY is set.")
else:
    print("❌ GROQ_API_KEY is NOT set.")

if tavily_key:
    print("✅ TAVILY_API_KEY is set.")
else:
    print("❌ TAVILY_API_KEY is NOT set.")


✅ GROQ_API_KEY is set.
✅ TAVILY_API_KEY is set.


## Provide User Input

In [18]:
# ==== User Inputs ====
skills = input("Enter your skills (comma-separated): ")
experience = input("Enter your experience (e.g., 3 years): ")
location = input("Enter your preferred location: ")
job_type = input("Enter job type (full-time, part-time, remote): ")

Enter your skills (comma-separated): python, sql, machine learning
Enter your experience (e.g., 3 years): 2
Enter your preferred location: pune
Enter job type (full-time, part-time, remote): full-time


## Providing prompt for LLM

In [19]:
# ==== Format Query for LLM ====
prompt_template = PromptTemplate(
    input_variables=["skills", "experience", "location", "job_type"],
    template=(
        "Find recent job postings relevant to the following profile:\n"
        "- Skills: {skills}\n"
        "- Experience: {experience}\n"
        "- Preferred location: {location}\n"
        "- Job type: {job_type}\n"
        "Return job title, company, location, and link in a concise format."
    )
)
query = prompt_template.format(
    skills=skills,
    experience=experience,
    location=location,
    job_type=job_type
)

## Retrieving Jobs from Browser

In [21]:
# ==== Try Groq LLM Retrieval ====
try:
    llm = ChatGroq(model="openai/gpt-oss-20b", temperature=0.7, api_key=os.environ["GROQ_API_KEY"])
    groq_response = llm.invoke(query)
    print("\n--- Groq LLM Job Search Results ---")
    print(groq_response)
except Exception as e:
    print(f"[Groq Error]: {e}")
    groq_response = None

# ==== If Groq Fails, Try Tavily API ====
if not groq_response:
    try:
        search_query = f"{job_type} jobs for {skills} in {location} with {experience} experience"
        url = "https://api.tavily.com/search"
        payload = {
            "api_key": os.environ["TAVILY_API_KEY"],
            "query": search_query,
            "search_depth": "basic",
            "max_results": 5
        }
        response = requests.post(url, json=payload)
        data = response.json()

        print("\n--- Tavily Job Search Results ---")
        for item in data.get("results", []):
            print(f"{item.get('title')} - {item.get('url')}")
    except Exception as e:
        print(f"[Tavily Error]: {e}")

[Groq Error]: Error code: 500 - {'error': {'message': 'Internal Server Error', 'type': 'internal_server_error'}}

--- Tavily Job Search Results ---
63 machine learning algorithm developer jobs in Pune, July 2025 - https://www.glassdoor.co.in/Job/pune-machine-learning-algorithm-developer-jobs-SRCH_IL.0,4_IC2856202_KO5,41.htm
5,000 Sql, Python Job Vacancies in Pune, Maharashtra | Indeed - https://in.indeed.com/q-sql,-python-l-pune,-maharashtra-jobs.html
python,sql,data science jobs in pune, maharashtra - Indeed - https://in.indeed.com/q-python,sql,data-science-l-pune,-maharashtra-jobs.html
Machine Learning Engineer Jobs In Pune-maharashtra - Naukri.com - https://www.naukri.com/machine-learning-engineer-jobs-in-pune-maharashtra-136
27865 Python Job Vacancies In Pune - Naukri.com - https://www.naukri.com/python-jobs-in-pune
